# Investigating a dataset - Udacity's Nano Degree Program

As part of the project, I have selected the curated movie database. I will try to clean, analyze, interpret and answer few questions for which the answer can be found within this particular dataset.

## Four step analysis
- Introduction - getting the initial information about the dataset (number of NAs/0s, a snapshot of the initial statistics about the dataset etc.)
- Data Cleaning - removing duplicate rows, removing movies with budget and/or revenue of 0 as we cannot infer any analysis on those
- Data Analysis - trying to answer the questions posed below.
- Conclusions

Questions:
- Which was the most profitable movie?
- Which was the most profitable movie per genre? What about per year? 
- What year was the most profitable? Both in initial values and adjusted.
- How does the top 10 most profitable movies change when using the adjusted revenues (they are adjusted for inflation as of 2010 dollars)
- What are the main correlations between profits and other variables? Popularity, Director, Runtime, Genre, Production Company, Voting Rating? - including visuals
- Is there a recipe for success? Based on the correlation findings, can we derive an approach to get a profitable movie?


I will firstly import what I think the main libraries needed for this project. I will add more on the way as I see fit and the changes will be reflected as I go along.

### Introduction

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline # To have the graphs displayed insine the workbook
import seaborn as sns

UsageError: Line magic function `%` not found.


In [23]:
df = pd.read_csv('tmdb-movies.csv') # Reading the data from a csv file using the pandas functionality

In [24]:
df.shape # 10866 rows by 21 columns

(10866, 21)

In [25]:
df.head(1) # In order to see the first snapshot of the data

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09


In [26]:
df.info() # Concise summary of type of values in the columns and how many

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
id                      10866 non-null int64
imdb_id                 10856 non-null object
popularity              10866 non-null float64
budget                  10866 non-null int64
revenue                 10866 non-null int64
original_title          10866 non-null object
cast                    10790 non-null object
homepage                2936 non-null object
director                10822 non-null object
tagline                 8042 non-null object
keywords                9373 non-null object
overview                10862 non-null object
runtime                 10866 non-null int64
genres                  10843 non-null object
production_companies    9836 non-null object
release_date            10866 non-null object
vote_count              10866 non-null int64
vote_average            10866 non-null float64
release_year            10866 non-null int64
budget_adj              1

In [27]:
df.describe() # Initial snapshot of statistics about the data

,id,popularity,budget,revenue,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj
count,10866.000000,10866.000000,1.086600e+04,1.086600e+04,10866.000000,10866.000000,10866.000000,10866.000000,1.086600e+04,1.086600e+04
mean,66064.177434,0.646441,1.462570e+07,3.982332e+07,102.070863,217.389748,5.974922,2001.322658,1.755104e+07,5.136436e+07
std,92130.136561,1.000185,3.091321e+07,1.170035e+08,31.381405,575.619058,0.935142,12.812941,3.430616e+07,1.446325e+08
min,5.000000,0.000065,0.000000e+00,0.000000e+00,0.000000,10.000000,1.500000,1960.000000,0.000000e+00,0.000000e+00
25%,10596.250000,0.207583,0.000000e+00,0.000000e+00,90.000000,17.000000,5.400000,1995.000000,0.000000e+00,0.000000e+00
50%,20669.000000,0.383856,0.000000e+00,0.000000e+00,99.000000,38.000000,6.000000,2006.000000,0.000000e+00,0.000000e+00
75%,75610.000000,0.713817,1.500000e+07,2.400000e+07,111.000000,145.750000,6.600000,2011.000000,2.085325e+07,3.369710e+07
max,417859.000000,32.985763,4.250000e+08,2.781506e+09,900.000000,9767.000000,9.200000,2015.000000,4.250000e+08,2.827124e+09


There seems to be 0 values for budget, revenue, runtime, budj adj and rev adj. Let's see how many.

In [28]:
df.isna().sum()

id                         0
imdb_id                   10
popularity                 0
budget                     0
revenue                    0
original_title             0
cast                      76
homepage                7930
director                  44
tagline                 2824
keywords                1493
overview                   4
runtime                    0
genres                    23
production_companies    1030
release_date               0
vote_count                 0
vote_average               0
release_year               0
budget_adj                 0
revenue_adj                0
dtype: int64

In [31]:
df.fillna(0, inplace=True)

In [34]:
df.isna().sum() # Expected result

id                      0
imdb_id                 0
popularity              0
budget                  0
revenue                 0
original_title          0
cast                    0
homepage                0
director                0
tagline                 0
keywords                0
overview                0
runtime                 0
genres                  0
production_companies    0
release_date            0
vote_count              0
vote_average            0
release_year            0
budget_adj              0
revenue_adj             0
dtype: int64

### Data Cleaning 
- Remove any duplicated rows
- Remove movies with budget and/or revenue of 0
- Remove irrelevant columns

In [38]:
len(df) - len(df.drop_duplicates()) # Drop duplicates will return the value after dropping any duplicated rows. This way we can the number of dupes

1

In [47]:
df.drop_duplicates(inplace=True)
print ('After removing the duplicated rows, we should expect ' + str(df.shape[0]) + ' rows and ' + str(df.shape[1]) + ' columns')

After removing the duplicated rows, we should expect 10865 rows and 21 columns


In [48]:
df.drop(['imdb_id', 'homepage', 'tagline', 'keywords', 'overview',], axis = 1, inplace=True) # Dropping few columns that I find irrelevant to the analysis 

In [50]:
df.info() # As expected

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10865 entries, 0 to 10865
Data columns (total 16 columns):
id                      10865 non-null int64
popularity              10865 non-null float64
budget                  10865 non-null int64
revenue                 10865 non-null int64
original_title          10865 non-null object
cast                    10865 non-null object
director                10865 non-null object
runtime                 10865 non-null int64
genres                  10865 non-null object
production_companies    10865 non-null object
release_date            10865 non-null object
vote_count              10865 non-null int64
vote_average            10865 non-null float64
release_year            10865 non-null int64
budget_adj              10865 non-null float64
revenue_adj             10865 non-null float64
dtypes: float64(4), int64(6), object(6)
memory usage: 1.7+ MB
